In [11]:
import pandas as pd
import numpy as np
miscon_df = pd.read_csv('misconception_mapping.csv')
test_df = pd.read_csv('test_data.csv')
test_df = test_df.sort_values(by='QuestionId')
miscon_df['CorrectPred'] = 0
miscon_df['AppearInTest'] = 0
miscon_df['Predicted'] = 0


In [2]:
#load predictions
res_df2 = pd.read_parquet('df_submission2.parquet', engine='pyarrow')


In [6]:
#process pred df
#count_top_n for recall and precision
def process_pred(pred_df, test_df, count_top_n):
    pred_df[['QuestionId', 'Answer']] = pred_df['QuestionId_Answer'].str.split('_', expand=True)

    target = []
    pred = []
    for _, row in pred_df.iterrows():
        question_id = row['QuestionId']
        answer = row['Answer']
        if int(question_id) in test_df['QuestionId'].values:
            misconception_column = f'Misconception{answer}Id'
            r = test_df[test_df['QuestionId'] == int(question_id)]
            targ = int(r[misconception_column].values[0])
            target.append(targ)
            pred.append(row['MisconceptionId'])
    calc_recall_precision(target, pred, count_top_n)
    mapk_score = mapk(target, pred)
    print(f'mapk score: {mapk_score}')

#calculate recall and precision
def calc_recall_precision(target, pred, count_top_n):
    #print(pred)
    for i in range(len(target)):
        #check top 5 of pred, if any one is correct, it's counted
        pred25 = [int(num) for num in pred[i].split()]
        for j in range(count_top_n):
            if j == 0:
                miscon_df.loc[target[i], 'AppearInTest'] += 1
                miscon_df.loc[pred25[j], 'Predicted'] += 1
            if target[i] == pred25[j]:
                miscon_df.loc[target[i], 'CorrectPred'] += 1
                break
    total_recall = 0
    recall_len = 0
    total_precision = 0
    precision_len = 0
    for i, row in miscon_df.iterrows():
        if row['AppearInTest'] != 0:
            total_recall += row['CorrectPred'] / row['AppearInTest']
            recall_len += 1
        if row['Predicted'] != 0:
            total_precision += row['CorrectPred'] / row['Predicted']
            precision_len += 1
    recall = total_recall / recall_len
    precision = total_precision / precision_len
    print(f'recall: {recall}')
    print(f'precision: {precision}')

def apk(actual, predicted, k=25):
    if not actual:
        return 0.0

    actual = [actual]
    #comment below line if predicted is already a list
    predicted = list(map(int, predicted.split()))

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

#from baseline.py
def mapk(actual, predicted, k=25):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [ ]:
#pd.options.display.max_rows = None
#print(test_df['SubjectName'].value_counts())

#88 questions in test_df
algebra_topics = ['Place Value', 'Expanding Single Brackets', 'BIDMAS',  'Multiplying and Dividing with Decimals', 'Adding and Subtracting with Decimals', 'Adding and Subtracting Negative Numbers', 'Converting between Fractions and Percentages', 'Mental Multiplication and Division', 'Linear Sequences (nth term)', 'Substitution into Formula', 'Writing Expressions', 'Other Sequences', 'Ordering Negative Numbers', 'Equivalent Fractions', 'Adding and Subtracting Fractions', 'Multiples and Lowest Common Multiple', 'Rounding to Decimal Places', 'Factors and Highest Common Factor', 'Expanding Double Brackets', 'Simplifying Fractions', 'Writing Ratios', 'Written Division', 'Sequences-Others', 'Simplifying Algebraic Fractions', 'Fractions of an Amount', 'Simultaneous Equations', 'Ordering Fractions', 'Expanding Triple Brackets and more', 'Multiplying Terms', 'Indirect (Inverse) Proportion', 'Dividing Fractions', 'implifying Expressions by Collecting Like Terms', 'Converting Mixed Number and Improper Fractions', 'Operations with Surds', 'Written Multiplication', 'Multiplying Fractions', 'Algebraic Proof', 'Rounding to Significant Figures', 'Factorising into a Double Bracket', 'Factorising into a Single Bracket']

#28 questions in test_df
linear_algebra_topics = ['Distance Between Two Co-ordinates', 'Graphical Solution of Simultaneous Equations', 'Plotting Lines from Tables of Values', 'Solving Linear Inequalities', 'Plotting Quadratics from Tables of Values', 'Quadratic Equations', 'Linear Equations', 'Function Machines', 'Reflection', 'Quadratic Graphs-Others', 'Translation and Vectors', 'Midpoint Between Two Co-ordinates', 'Finding the Gradient and Intercept of a Line from the Equation']

#38 questions in test_df
geometry_topics = ['Basic Angle Facts (straight line, opposite, around a point, etc)', 'Real Life Graphs', 'Volume and Capacity Units', 'Area of Simple Shapes', 'Squares, Cubes, etc', 'Names and Properties of 3D Shapes', 'Nets', 'Length Scale Factors in Similar Shapes', 'Measuring Angles', 'Missing Lengths', 'Construct Triangle', 'Properties of Polygons', 'Parts of a Circle', 'Volume of Non-Prisms', 'Co-ordinate Geometry with Straight Lines', 'Parallel Lines', 'Perimeter', 'Properties of Triangles', 'Right-angled Triangles (SOHCAHTOA)', 'Properties of Quadrilaterals']

#11 questions in test_df
other_topics = ['Estimation', 'Time', 'Types, Naming and Estimating', 'Trial and Improvement and Iterative Methods', 'Speed, Distance, Time', 'Systematic Listing Strategies']

#13 questions in test_df
stats_topics = ['Pie Chart', 'Averages (mean, median, mode) from a List of Data', 'Probability of Single Events', 'Pictogram', 'Time Series and Line Graphs', 'Venn Diagrams', 'Range and Interquartile Range from a List of Data', 'Averages and Range from Grouped Data']

In [8]:
algebra_df = test_df[test_df['SubjectName'].isin(algebra_topics)]
#print(len(algebra_df))
la_df = test_df[test_df['SubjectName'].isin(linear_algebra_topics)]
#print(len(la_df))
geo_df = test_df[test_df['SubjectName'].isin(geometry_topics)]
#print(len(geo_df))
ot_df = test_df[test_df['SubjectName'].isin(other_topics)]
#print(len(ot_df))
stats_df = test_df[test_df['SubjectName'].isin(stats_topics)]
#print(len(stats_df))

In [12]:
print('algebra')
process_pred(res_df2, algebra_df, 5)
print('\nlinear algebra')
process_pred(res_df2, la_df, 5)
print('\ngeometry')
process_pred(res_df2, geo_df, 5)
print('\nothers')
process_pred(res_df2, ot_df, 5)
print('\nstats')
process_pred(res_df2, stats_df, 5)

algebra
recall: 0.22984749455337689
precision: 0.12990196078431374
mapk score: 0.1171275997319352

linear algebra
recall: 0.22717013888888893
precision: 0.1257396449704142
mapk score: 0.1316443826948029

geometry
recall: 0.2657475490196079
precision: 0.135844250363901
mapk score: 0.24562834929196628

others
recall: 0.27702546296296293
precision: 0.14256569847856151
mapk score: 0.29523809523809524

stats
recall: 0.28254219409282694
precision: 0.14775031685678075
mapk score: 0.22286295427599773


In [13]:
#all qns
process_pred(res_df2, test_df, 5)

recall: 0.28323108384458073
precision: 0.13833932853717024
mapk score: 0.16373559280668956
